In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [46]:
df = spark.read.parquet("data/final/merge_jugadores.parquet")
df.show(5)

+----------------+-------------+---------+-----------+------+---+------+----+---+---+-------+----+----+----+----+---+---+-------+-----+----------+------------+------------+------------+----+---+----+---+------+-----------+----------+----------+---+--------+---------+----------+---------------+-----------+----------------+---------+--------------+---+---+-----+----------------+---+-----+-------------+-------------+-------------+-------------+----------+--------+----------+---------+----------+-------+---------------+---------------+--------------------+-------------------+---+---+----+---+----+-------+----+----------------+------+----------+
|          Player|        Squad|   Season|Competition|  Born| MP|Starts| Min|Gls|Ast|YellowC|RedC|PrgC|PrgP|PrgR|Fls|Off|Crosses|Recov|Aerialwon%|Tack_Def_3rd|Tack_Mid_3rd|Tack_Att_3rd|Tkl%|Tkl|TklW|Int|Blocks|Block_Shots|Bolck_Pass|Clearences|Err|Pass_cmp|Pass_cmp%|Pass_Short|Pass_cmp_Short%|Pass_Medium|Pass_cmp_Medium%|pass_Long|Pass_cmp_Long%|xAG

In [26]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType


float_columns = [
    'Born','MP','Starts','Min','Gls','Ast','YellowC','RedC','PrgC','PrgP','PrgR',
    'Fls','Off','Crosses','Recov','Aerialwon%','Tack_Def_3rd','Tack_Mid_3rd','Tack_Att_3rd','Tkl%',
    'Tkl','TklW','Int','Blocks','Block_Shots','Bolck_Pass','Clearences','Err',
    'Pass_cmp','Pass_cmp%','Pass_cmp_Short%','Pass_cmp_Medium%','Pass_cmp_Long%',
    'xAG','xA','A-xAG','Pass_cmp_Att_3rd','PPA','CrsPA',
    'Touch_Def_3rd','Touch_Mid_3rd','Touch_Att_3rd','Touch_Att_Pen',
    'drib_Att','drib_Succ%','PrgDist','Carries_Att_3rd','Carries_Att_Pen',
    'fail_To_Gain_Control','Loss_Control_Tackle','Sh','SoT','G/Sh',
    'xG','npxG','npxG/Sh','G-xG','Height','Value'
]

for col_name in float_columns:
    df = df.withColumn(col_name, col(col_name).cast(FloatType()))
    
from pyspark.sql.functions import isnan, when, count

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in float_columns
]).show()


+----+---+------+---+---+---+-------+----+----+----+----+---+---+-------+-----+----------+------------+------------+------------+----+---+----+---+------+-----------+----------+----------+---+--------+---------+---------------+----------------+--------------+---+---+-----+----------------+---+-----+-------------+-------------+-------------+-------------+--------+----------+-------+---------------+---------------+--------------------+-------------------+---+---+----+---+----+-------+----+------+-----+
|Born| MP|Starts|Min|Gls|Ast|YellowC|RedC|PrgC|PrgP|PrgR|Fls|Off|Crosses|Recov|Aerialwon%|Tack_Def_3rd|Tack_Mid_3rd|Tack_Att_3rd|Tkl%|Tkl|TklW|Int|Blocks|Block_Shots|Bolck_Pass|Clearences|Err|Pass_cmp|Pass_cmp%|Pass_cmp_Short%|Pass_cmp_Medium%|Pass_cmp_Long%|xAG| xA|A-xAG|Pass_cmp_Att_3rd|PPA|CrsPA|Touch_Def_3rd|Touch_Mid_3rd|Touch_Att_3rd|Touch_Att_Pen|drib_Att|drib_Succ%|PrgDist|Carries_Att_3rd|Carries_Att_Pen|fail_To_Gain_Control|Loss_Control_Tackle| Sh|SoT|G/Sh| xG|npxG|npxG/Sh|G-xG|He

In [47]:
df_unidos = df.toPandas()

df_unidos.sort_values(by="Player").to_csv("data/unidos.csv", index=False)

df.columns

In [48]:
df.columns

['Player',
 'Squad',
 'Season',
 'Competition',
 'Born',
 'MP',
 'Starts',
 'Min',
 'Gls',
 'Ast',
 'YellowC',
 'RedC',
 'PrgC',
 'PrgP',
 'PrgR',
 'Fls',
 'Off',
 'Crosses',
 'Recov',
 'Aerialwon%',
 'Tack_Def_3rd',
 'Tack_Mid_3rd',
 'Tack_Att_3rd',
 'Tkl%',
 'Tkl',
 'TklW',
 'Int',
 'Blocks',
 'Block_Shots',
 'Bolck_Pass',
 'Clearences',
 'Err',
 'Pass_cmp',
 'Pass_cmp%',
 'Pass_Short',
 'Pass_cmp_Short%',
 'Pass_Medium',
 'Pass_cmp_Medium%',
 'pass_Long',
 'Pass_cmp_Long%',
 'xAG',
 'xA',
 'A-xAG',
 'Pass_cmp_Att_3rd',
 'PPA',
 'CrsPA',
 'Touch_Def_3rd',
 'Touch_Mid_3rd',
 'Touch_Att_3rd',
 'Touch_Att_Pen',
 'touch_Live',
 'drib_Att',
 'drib_Succ%',
 'Tckl_Drib',
 'Tckl_Drib%',
 'PrgDist',
 'Carries_Att_3rd',
 'Carries_Att_Pen',
 'fail_To_Gain_Control',
 'Loss_Control_Tackle',
 'Sh',
 'SoT',
 'G/Sh',
 'xG',
 'npxG',
 'npxG/Sh',
 'G-xG',
 'Pos',
 'Height',
 'Value']

In [31]:


import pandas as pd

df_fbref = pd.read_csv("data/limpios/stats/stats.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

df_fbref=df_fbref["Squad"]
df_mercado2=df_mercado["Squad"]


df_fbref.drop_duplicates().sort_values().to_csv("data/nombres_stat.csv", index=False)
df_mercado2.drop_duplicates().sort_values().to_csv("data/nombres_mercado.csv", index=False)

In [34]:
df_mercado.columns

Index(['Player', 'Pos', 'Squad', 'Height', 'Value', 'Season', 'Competition'], dtype='object')

In [35]:
df_mercado= df_mercado["Pos"].drop_duplicates()
df_mercado

0               Goalkeeper
4              Centre-Back
12               Left-Back
14              Right-Back
18      Defensive Midfield
21        Central Midfield
23      Attacking Midfield
26             Left Winger
29            Right Winger
30          Second Striker
32          Centre-Forward
159         Right Midfield
160          Left Midfield
4038            Midfielder
5297               Striker
Name: Pos, dtype: object

In [3]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

In [19]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

fbref = df_fbref["club_actual"]
understat = df_understat["club_actual"]
mercado = df_mercado["club_actual"]

nombre_fbref = fbref.dropna().sort_values().unique()
nombre_understat = understat.dropna().sort_values().unique()
nombre_mercado = mercado.dropna().sort_values().unique()

pd.Series(nombre_fbref).to_csv("prueba_data/club_actual_fbref.csv", index=False)
pd.Series(nombre_understat).to_csv("prueba_data/club_actual_understat.csv", index=False)
pd.Series(nombre_mercado).to_csv("prueba_data/club_actual_mercado.csv", index=False)

In [31]:
duplicados = df_fbref[df_fbref.duplicated(subset=["nombre"], keep=False)]
print(duplicados)
df_ordenado = duplicados.sort_values(by="nombre")
df_ordenado.to_csv("prueba_data/duplicados_fbref.csv", index=False)

                nombre Posicion  nacido  partidos_jugados  titularidades  \
0      paxten aaronson    MF,DF  2003.0                 7              0   
1       sargis adamyan    FW,MF  1993.0                24              4   
2        karim adeyemi    FW,MF  2002.0                24             20   
3           amine adli    FW,MF  2000.0                26             16   
4            felix agu    DF,FW  1999.0                 3              0   
...                ...      ...     ...               ...            ...   
6399   piotr zielinski       MF  1994.0                26              8   
6400      nadir zortea    DF,MF  1999.0                35             33   
6401  szymon zurkowski       MF  1997.0                 5              0   
6402        milan uric       FW  1990.0                18             13   
6403        milan uric       FW  1990.0                 9              3   

      minutos_jugados  goles  asistencias  amarillas  rojas  \
0                 173   

In [18]:
COLUMNAS_UTILES = {
    "fbref": [
        "Unnamed: 1_level_0_Player", "Unnamed: 4_level_0_Squad", "Unnamed: 3_level_0_Pos", "Unnamed: 6_level_0_Born", "Playing Time_MP", "Playing Time_Starts", "Playing Time_Min", "Performance_Gls", "Performance_Ast", "Performance_CrdY", "Performance_CrdR", "Progression_PrgC","Progression_PrgP","Progression_PrgR"
    ],
    "fbref_defense": [
       "Unnamed: 1_level_0_Player", "Unnamed: 4_level_0_Squad", "Tackles_Def 3rd", "Tackles_Mid 3rd","Tackles_Att 3rd", "Challenges_Tkl%", "Tackles_Tkl", "Tackles_TklW", "Int", "Blocks_Blocks", "Blocks_Sh", "Blocks_Pass", "Clr", "Err"
    ],
    "understat": [
        "nombre", "club_actual", "xG", "xA", "npxG", "shsots", "key_passes", "xGChain", "xGBuildup"
    ],
    "mercado": [
        "player_name", "current_club", "player_height_mtrs", "player_market_value_euro"
    ]
}

df_mercado = pd.read_csv("data/mercado/valores_mercado_LaLiga_2022-2023.csv")
mercado = ["player_name", "current_club", "player_height_mtrs", "player_market_value_euro"]
def limpiar_ligas_transfermarkt(df):
    ligas_principales = ["Serie A", "LaLiga", "Premier League", "Bundesliga"]
    return df[df["comp_name"].isin(ligas_principales)]

df_mercado = limpiar_ligas_transfermarkt(df_mercado)
columnas_utiles = COLUMNAS_UTILES.get("mercado", df_mercado.columns.tolist())
columnas_disponibles = [col for col in columnas_utiles if col in df_mercado.columns]
#df_mercado = df_mercado[mercado]
df_mercado.to_csv("prueba_data/mercadoLaLiga.csv", index=False)

In [37]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

col = ["nombre", "club_actual", "temporada", "liga"]

fbref= df_fbref[col].sort_values(by="nombre")
defense=df_fbref_defense[col].sort_values(by="nombre")
understat=df_understat[col].sort_values(by="nombre")
mercado=df_mercado[col].sort_values(by="nombre")

fbref.to_csv("prueba_data/limpios/fbref.csv", index=False)
defense.to_csv("prueba_data/limpios/defense.csv", index=False)
understat.to_csv("prueba_data/limpios/understat.csv", index=False)
mercado.to_csv("prueba_data/limpios/mercado.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'data/limpios/fbref/fbref.csv'

In [40]:
defense = pd.read_csv("data/possession/LaLiga_2022_2023_possession.csv")
defense.columns


Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', '90s',
       'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen',
       'Live', 'Att', 'Succ', 'Succ%', 'Tkld', 'Tkld%', 'Carries', 'TotDist',
       'PrgDist', 'PrgC', '1/3', 'CPA', 'Mis', 'Dis', 'Rec', 'PrgR',
       'Matches'],
      dtype='object')